Import Libraries and Dependancies

In [25]:
import glob
import os
import pandas as pd
import skimage.io
import skimage.color
from skimage.feature import hog
from skimage import exposure
from sklearn.model_selection import train_test_split
from sklearn import svm
from skimage.feature import graycomatrix,graycoprops
from skimage import io, color, exposure
import cv2
from skimage.measure import shannon_entropy
from scipy.stats import mode
import numpy as np

Load images corresponding to their path and label

In [26]:
csv_data = pd.read_csv('C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/train.csv',index_col=None)
new=csv_data.sort_values(by="name")
image_folder = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/train"
image_data={}
count=0
count_fail=0
for index, row in new.iterrows():
  img_name=(row.iloc[0])+".jpg"
  img_path = os.path.join(image_folder, img_name)
  img_label=row.iloc[1]
  if os.path.exists(img_path):
    image = skimage.io.imread(img_path)
    image_data[img_path] = {'image': image, 'label': img_label}
    count+=1
  else:
      count_fail+=1

count,count_fail




(4021, 0)

Load Test images (for future testing)

In [27]:
test_csv_data = pd.read_csv('C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/test.csv',index_col=None)
test_new=test_csv_data.sort_values(by="name")
test_image_folder = "C:/Users/sanja/OneDrive/WiSe_23/ISM/ism_project_2023/ism_project_2023/test"
test_image_data={}
count=0
count_fail=0
for index, row in test_new.iterrows():
  test_img_name=(row.iloc[0])+".jpg"
  test_img_path = os.path.join(test_image_folder, test_img_name)
  test_img_label=row.iloc[1]
  if os.path.exists(img_path):
    test_image = skimage.io.imread(test_img_path)
    test_image_data[test_img_path] = {'image': test_image, 'label': test_img_label}
    count+=1
  else:
      count_fail+=1

count,count_fail



(2139, 0)

Helper to return metrics

In [28]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
def get_metrics(labels, predictions):
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')
    precision = precision_score(labels, predictions, average='macro')
    return accuracy, f1, recall, precision

Gray Level Co-occurence Matrix (GLCM) with Mean Shift Filtering

In [31]:
for img_path,data in image_data.items():
  image=data["image"]
  #filtered_image = cv2.pyrMeanShiftFiltering(image, sp=15, sr=30)
  image = skimage.filters.unsharp_mask(image, radius=2, amount=5)
  image=image.astype(np.uint8)
  gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  GLCM = graycomatrix(gray_segmented_image, [1], [0])
  GLCM_Energy = graycoprops(GLCM, 'energy')[0,0]
  data['Energy'] = GLCM_Energy
  GLCM_corr = graycoprops(GLCM, 'correlation')[0,0]
  data['Corr'] = GLCM_corr
  GLCM_diss = graycoprops(GLCM, 'dissimilarity')[0,0]
  data['Diss_sim'] = GLCM_diss
  GLCM_hom = graycoprops(GLCM, 'homogeneity')[0,0]
  data['Homogen'] = GLCM_hom
  GLCM_contr = graycoprops(GLCM, 'contrast')[0,0]
  data['Contrast'] = GLCM_contr

  GLCM2 = graycomatrix(gray_segmented_image, [3], [0])       
  GLCM_Energy2 = graycoprops(GLCM2, 'energy')[0,0]
  data['Energy2'] = GLCM_Energy2
  GLCM_corr2 = graycoprops(GLCM2, 'correlation')[0,0]
  data['Corr2'] = GLCM_corr2       
  GLCM_diss2 = graycoprops(GLCM2, 'dissimilarity')[0,0]
  data['Diss_sim2'] = GLCM_diss2       
  GLCM_hom2 = graycoprops(GLCM2, 'homogeneity')[0,0]
  data['Homogen2'] = GLCM_hom2       
  GLCM_contr2 = graycoprops(GLCM2, 'contrast')[0,0]
  data['Contrast2'] = GLCM_contr2

  GLCM3 = graycomatrix(gray_segmented_image, [5], [0])       
  GLCM_Energy3 = graycoprops(GLCM3, 'energy')[0,0]
  data['Energy3'] = GLCM_Energy3
  GLCM_corr3 = graycoprops(GLCM3, 'correlation')[0,0]
  data['Corr3'] = GLCM_corr3       
  GLCM_diss3 = graycoprops(GLCM3, 'dissimilarity')[0,0]
  data['Diss_sim3'] = GLCM_diss3       
  GLCM_hom3 = graycoprops(GLCM3, 'homogeneity')[0,0]
  data['Homogen3'] = GLCM_hom3       
  GLCM_contr3 = graycoprops(GLCM3, 'contrast')[0,0]
  data['Contrast3'] = GLCM_contr3

  GLCM4 = graycomatrix(gray_segmented_image, [0], [np.pi/4])       
  GLCM_Energy4 = graycoprops(GLCM4, 'energy')[0,0]
  data['Energy4'] = GLCM_Energy4
  GLCM_corr4 = graycoprops(GLCM4, 'correlation')[0,0]
  data['Corr4'] = GLCM_corr4       
  GLCM_diss4 = graycoprops(GLCM4, 'dissimilarity')[0,0]
  data['Diss_sim4'] = GLCM_diss4       
  GLCM_hom4 = graycoprops(GLCM4, 'homogeneity')[0,0]
  data['Homogen4'] = GLCM_hom4       
  GLCM_contr4 = graycoprops(GLCM4, 'contrast')[0,0]
  data['Contrast4'] = GLCM_contr4
  
  GLCM5 = graycomatrix(gray_segmented_image, [0], [np.pi/2])       
  GLCM_Energy5 = graycoprops(GLCM5, 'energy')[0,0]
  data['Energy5'] = GLCM_Energy5
  GLCM_corr5 = graycoprops(GLCM5, 'correlation')[0,0]
  data['Corr5'] = GLCM_corr5       
  GLCM_diss5 = graycoprops(GLCM5, 'dissimilarity')[0,0]
  data['Diss_sim5'] = GLCM_diss5       
  GLCM_hom5 = graycoprops(GLCM5, 'homogeneity')[0,0]
  data['Homogen5'] = GLCM_hom5       
  GLCM_contr5 = graycoprops(GLCM5, 'contrast')[0,0]
  data['Contrast5'] = GLCM_contr5


HU Moments


In [32]:
for img_path,data in image_data.items():
    image=data["image"]
    image = skimage.filters.unsharp_mask(image, radius=2, amount=5)
    image=image.astype(np.uint8)
    gray_segmented_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, image = cv2.threshold(gray_segmented_image*255, 128, 255, cv2.THRESH_BINARY)
    moments = cv2.moments(image)
    hu_moments= cv2.HuMoments(moments)
    data["Hu_1"]=hu_moments[0][0]
    data["Hu_2"]=hu_moments[1][0]
    data["Hu_3"]=hu_moments[2][0]
    data["Hu_4"]=hu_moments[3][0]
    data["Hu_5"]=hu_moments[4][0]
    data["Hu_6"]=hu_moments[5][0]
    data["Hu_7"]=hu_moments[6][0]
    

Statistical Features

In [33]:
from scipy.stats import skew, kurtosis
for img_path,data in image_data.items():
  image=data["image"]
  image = skimage.filters.unsharp_mask(image, radius=2, amount=5)
  data['mean_value']= np.mean(image)
  data['std_deviation'] = np.std(image)
  data['entropy_value'] = shannon_entropy(image)
  data["median"] = np.median(image)
  data["percentile_25"] = np.percentile(image, 25)
  data["percentile_75"] = np.percentile(image, 75)
  data["skewness"] = skew(image, axis=None)
  data["kurtosis"] = kurtosis(image, axis=None)


Scale and Pre Process Training and Test data for Model 

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

feature_vectors = []

for img_path, data in image_data.items():
    glcm_features = [data['Energy'], data['Corr'], data['Diss_sim'], data['Homogen'], data['Contrast'],
                     data['Energy2'], data['Corr2'], data['Diss_sim2'], data['Homogen2'], data['Contrast2'],
                     data['Energy3'], data['Corr3'], data['Diss_sim3'], data['Homogen3'], data['Contrast3'],
                     data['Energy4'], data['Corr4'], data['Diss_sim4'], data['Homogen4'], data['Contrast4'],
                     data['Energy5'], data['Corr5'], data['Diss_sim5'], data['Homogen5'], data['Contrast5']]
    statistical_features = [data['mean_value'], data['std_deviation'], data['entropy_value'], data['median'], data['percentile_25'], data['percentile_75']]
    hu_features = [data["Hu_1"],data["Hu_2"],data["Hu_3"],data["Hu_4"],data["Hu_5"],data["Hu_6"],data["Hu_7"]]
    combined_features = glcm_features + statistical_features+hu_features
    feature_vectors.append(combined_features)


In [35]:

labels = [data['label'] for img_path, data in image_data.items()]

X_train, X_test, y_train, y_test = train_test_split(feature_vectors, labels, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
type(y_train)

list

In [36]:
from flaml import AutoML
automl = AutoML()
automl.fit(X_train_scaled, np.array(y_train), task="classification",estimator_list=['rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1'])

[flaml.automl.logger: 12-13 23:07:55] {1679} INFO - task = classification
[flaml.automl.logger: 12-13 23:07:55] {1690} INFO - Evaluation method: cv
[flaml.automl.logger: 12-13 23:07:55] {1788} INFO - Minimizing error metric: log_loss
[flaml.automl.logger: 12-13 23:07:55] {1811} WARNING - No search budget is provided via time_budget or max_iter. Training only one model per estimator. Zero-shot AutoML is used for certain tasks and estimators. To tune hyperparameters for each estimator, please provide budget either via time_budget or max_iter.
[flaml.automl.logger: 12-13 23:07:55] {1900} INFO - List of ML learners in AutoML Run: ['rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'lrl1']
[flaml.automl.logger: 12-13 23:07:55] {2218} INFO - iteration 0, current learner rf
[flaml.automl.logger: 12-13 23:08:23] {2344} INFO - Estimated sufficient time budget=10000s. Estimated necessary time budget=10s.
[flaml.automl.logger: 12-13 23:08:23] {2391} INFO -  at 28.5s,	estimator rf's best error=0.993

c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.logger: 12-13 23:14:20] {2391} INFO -  at 385.0s,	estimator lrl1's best error=1.1026,	best estimator xgboost's best error=0.9392


c:\Users\sanja\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[flaml.automl.logger: 12-13 23:15:28] {2627} INFO - retrain xgboost for 68.1s
[flaml.automl.logger: 12-13 23:15:28] {2630} INFO - retrained model: XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=0.8815698870579244, colsample_bynode=None,
              colsample_bytree=0.7057137578225323, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints=None, learning_rate=0.002432260930606481,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=0, max_leaves=4,
              min_child_weight=0.7673654415794792, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=8315,
              n_jobs=-1, num_parallel_tree=None, objective='multi:softprob', ...)
[flaml.automl.log

In [ ]:
automl.best_estimator,

('xgboost',)

In [38]:
automl.best_config

{'n_estimators': 8315,
 'max_leaves': 4,
 'min_child_weight': 0.7673654415794792,
 'learning_rate': 0.002432260930606481,
 'subsample': 0.8476000618302348,
 'colsample_bylevel': 0.8815698870579244,
 'colsample_bytree': 0.7057137578225323,
 'reg_alpha': 0.0016838090603716895,
 'reg_lambda': 0.28815989841009226}

Support Vector Machine

In [39]:
from sklearn.svm import SVC

svm_classifier = SVC(kernel="rbf", random_state=42, C=20, gamma=0.1)

svm_classifier.fit(X_train_scaled, y_train)

accuracy = svm_classifier.score(X_test_scaled, y_test)
test_predictions = svm_classifier.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- SVM --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))



--- SVM --- 
Accuracy:  66 
F1:  58 
Recall:  59 
Precision:  61


In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
cross_val_score(svm.SVC(kernel="rbf",C=20,gamma=0.1), X_train_scaled, y_train, cv=5, scoring="accuracy")

array([0.64130435, 0.65474339, 0.64696734, 0.61897356, 0.64696734])

In [ ]:
from sklearn.model_selection import GridSearchCV
clf=GridSearchCV(svm.SVC(),{"C":[1,5,10,15,20],"kernel":["rbf","linear","sigmoid"]},cv=5,return_train_score=False)
clf.fit(X_train_scaled,y_train)
clf.cv_results_
cldf=pd.DataFrame(clf.cv_results_)

In [ ]:
cldf

mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.161413      0.008195         0.078729        0.001859       1   
1        0.222035      0.024510         0.017495        0.001606       1   
2        0.186374      0.006544         0.036667        0.003033       1   
3        0.152372      0.005523         0.071597        0.000861       5   
4        0.407422      0.013119         0.015887        0.000553       5   
5        0.141487      0.006325         0.031847        0.001672       5   
6        0.156592      0.010296         0.071049        0.001995      10   
7        0.628125      0.019183         0.015416        0.000650      10   
8        0.138213      0.004117         0.034112        0.004224      10   
9        0.158082      0.003030         0.068322        0.003228      15   
10       0.825496      0.030455         0.016235        0.001403      15   
11       0.139449      0.003339         0.030433        0.002798      15   
12       0.170480      0.003991         0.069258        0.001477      20   
13       1.035169      0.032207         0.017173        0.000534      20   
14       0.141967      0.005981         0.032525        0.001789      20   

   param_kernel                          params  split0_test_score  \
0           rbf       {'C': 1, 'kernel': 'rbf'}           0.548137   
1        linear    {'C': 1, 'kernel': 'linear'}           0.503106   
2       sigmoid   {'C': 1, 'kernel': 'sigmoid'}           0.352484   
3           rbf       {'C': 5, 'kernel': 'rbf'}           0.590062   
4        linear    {'C': 5, 'kernel': 'linear'}           0.534161   
5       sigmoid   {'C': 5, 'kernel': 'sigmoid'}           0.354037   
6           rbf      {'C': 10, 'kernel': 'rbf'}           0.613354   
7        linear   {'C': 10, 'kernel': 'linear'}           0.546584   
8       sigmoid  {'C': 10, 'kernel': 'sigmoid'}           0.350932   
9           rbf      {'C': 15, 'kernel': 'rbf'}           0.624224   
10       linear   {'C': 15, 'kernel': 'linear'}           0.548137   
11      sigmoid  {'C': 15, 'kernel': 'sigmoid'}           0.336957   
12          rbf      {'C': 20, 'kernel': 'rbf'}           0.625776   
13       linear   {'C': 20, 'kernel': 'linear'}           0.554348   
14      sigmoid  {'C': 20, 'kernel': 'sigmoid'}           0.336957   

    split1_test_score  split2_test_score  split3_test_score  \
0            0.576983           0.573872           0.578538   
1            0.552100           0.547434           0.544323   
2            0.413686           0.410575           0.376361   
3            0.606532           0.617418           0.622084   
4            0.566096           0.569207           0.558320   
5            0.356143           0.377916           0.363919   
6            0.611198           0.629860           0.629860   
7            0.569207           0.573872           0.564541   
8            0.362364           0.353033           0.360809   
9            0.623639           0.646967           0.629860   
10           0.576983           0.567652           0.562986   
11           0.357698           0.376361           0.360809   
12           0.632970           0.657854           0.640747   
13           0.573872           0.569207           0.569207   
14           0.356143           0.370140           0.363919   

    split4_test_score  mean_test_score  std_test_score  rank_test_score  
0            0.587869         0.573080        0.013316                5  
1            0.524106         0.534214        0.018258               10  
2            0.388802         0.388382        0.022655               11  
3            0.618974         0.611014        0.011713                4  
4            0.558320         0.557221        0.012301                9  
5            0.379471         0.366297        0.010655               12  
6            0.632970         0.623448        0.009218                3  
7            0.555210         0.561883        0.009829           

In [ ]:
clf.best_score_,clf.best_params_,clf.best_estimator_

(0.6393076900785333, {'C': 20, 'kernel': 'rbf'}, SVC(C=20))

In [41]:
from xgboost import XGBClassifier

bst = XGBClassifier(n_estimators=8315,learning_rate=0.8, booster='gbtree', max_depth=6, min_child_weight=0.7673654415794792, gamma=0.1, subsample=0.8476000618302348, colsample_bytree=0.7057137578225323, colsample_bylevel=0.8815698870579244, reg_alpha=0.0016838090603716895, reg_lambda=0.28815989841009226, random_state=42, n_jobs=-1, verbosity=0)
bst.fit(X_train_scaled, y_train)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=0.8815698870579244, colsample_bynode=None,
              colsample_bytree=0.7057137578225323, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.1, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.8, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=0.7673654415794792, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=8315,
              n_jobs=-1, num_parallel_tree=None, objective='multi:softprob', ...)

In [42]:
X=bst.score(X_test_scaled, y_test)
test_predictions = bst.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- XGB --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))


--- XGB --- 
Accuracy:  63 
F1:  57 
Recall:  57 
Precision:  57


In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis(solver='svd')

lda.fit(X_train_scaled, y_train)
test_predictions = lda.predict(X_test_scaled)
accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- LDA --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))


--- LDA --- 
Accuracy:  59 
F1:  50 
Recall:  51 
Precision:  51


In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_classifier = MLPClassifier(hidden_layer_sizes=(150, 100, 50), activation='relu', solver='adam', max_iter=1000, batch_size=200)
mlp_classifier.out_activation_ = 'softmax'

mlp_classifier.fit(X_train_scaled, y_train)
test_predictions = mlp_classifier.predict(X_test_scaled)

accuracy, f1, recall, precision = get_metrics(y_test, test_predictions)
print('--- MLP --- \nAccuracy: ', round(accuracy*100), '\nF1: ', round(f1*100), '\nRecall: ', round(recall*100), '\nPrecision: ', round(precision*100))

--- MLP --- 
Accuracy:  65 
F1:  58 
Recall:  59 
Precision:  60
